In [1]:
import numpy as np
import pandas as pd

from ols_bootstrap.pairs import PairsBootstrap
from ols_bootstrap.residual import ResidualBootstrap
from ols_bootstrap.wild import WildBootstrap

pd.options.display.float_format = '{:20,.5f}'.format  ### Setting pd to have a numerical precision up to 5 decimal points

In [2]:
df = pd.read_csv('./balance2018.csv')
df = df[df['sales_clean'] != 0]
df = df[['sales_clean', 'tanass_clean', 'tax']]
df = df.dropna(subset=['tanass_clean', 'tax'])

df_scaled = df.applymap(lambda x: np.log(x + 1))

/home/phamv/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (66) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
df_sample = df_scaled.sample(n=10000, replace=False, random_state=42)

Y_data = pd.DataFrame(df_sample.iloc[:, 0])
X_data = pd.DataFrame(df_sample.iloc[:, 1:])

## Default SE on the original OLS is HC3, default CI on bootstrapped parameter is BC. 

That is by default se_type = 'hc3', ci_type = 'bc'.

The default seed value is None. For reproducability, use an integer of your choice. 

In [4]:
psb = PairsBootstrap(Y_data, X_data, reps = 1000, se_type='hc3', ci_type = 'bc', seed = 42)  # seed = None, se_type = 'hc3' and ci_type = 'bc' are the default options for these arguments.
psb.fit()

In [5]:
rsb = ResidualBootstrap(Y_data, X_data, reps = 1000, seed = 42)
rsb.fit()

In [6]:
wb_unif = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "uniform", seed = 42)
wb_unif.fit()

In [7]:
wb_stdn = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "standard_normal", seed = 42)
wb_stdn.fit()

In [8]:
wb_rad = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "rademacher", seed = 42)
wb_rad.fit()

In [9]:
wb_webb4 = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "webb4", seed = 42)
wb_webb4.fit()

In [10]:
wb_webb6 = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "webb6", seed = 42)
wb_webb6.fit()

In [11]:
wb_cont_mam = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "mammen_cont", seed = 42)
wb_cont_mam.fit()

In [12]:
wb_mam = WildBootstrap(Y_data, X_data, reps = 1000, from_distro = "mammen", seed = 42)
wb_mam.fit()

In [13]:
# Now: np.ndarray was supported as an input array. pd.Series, list, and tuples were also added to the supported list for an input array. 
# Before that only pd.DataFrame had been supported. 
X_data_np = X_data.to_numpy()
Y_data_np = Y_data.to_numpy()

In [14]:
wb_mam_np = WildBootstrap(Y_data_np, X_data_np, reps = 1000, from_distro = "mammen", seed = 42)
wb_mam_np.fit()

In [15]:
psb.summary()

+---------------------------------------------------------------------------------------------------------------------+
|               Pairs Bootstrap results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI               |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|     Var      | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|    const     |   6.2070   |     6.2070    | 0.0000 |     0.0454    |    0.0462    |    -1.70     | [6.1273, 6.3045] |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| tanass_clean |   0.1693   |     0.1691    | 0.0002 |     0.0050    |    0.0051    |    -1.56     | [0.1591, 0.1785] |
+--------------+------------+-----------

In [16]:
rsb.summary()

+---------------------------------------------------------------------------------------------------------------------+
|              Residual Bootstrap results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI             |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|     Var      | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|    const     |   6.2070   |     6.2058    | 0.0012 |     0.0454    |    0.0349    |    23.00     | [6.1420, 6.2788] |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| tanass_clean |   0.1693   |     0.1694    | 0.0001 |     0.0050    |    0.0039    |    22.09     | [0.1617, 0.1772] |
+--------------+------------+-----------

In [17]:
wb_unif.summary()

+---------------------------------------------------------------------------------------------------------------------+
|         Wild Bootstrap with Uniform results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI         |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|     Var      | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|    const     |   6.2070   |     6.2082    | 0.0013 |     0.0454    |    0.0447    |     1.48     | [6.1200, 6.2935] |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| tanass_clean |   0.1693   |     0.1693    | 0.0000 |     0.0050    |    0.0051    |    -3.06     | [0.1598, 0.1794] |
+--------------+------------+-----------

In [18]:
wb_stdn.summary()

+---------------------------------------------------------------------------------------------------------------------+
|     Wild Bootstrap with Standard Normal results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI     |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|     Var      | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|    const     |   6.2070   |     6.2059    | 0.0011 |     0.0454    |    0.0478    |    -5.42     | [6.1210, 6.3076] |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| tanass_clean |   0.1693   |     0.1689    | 0.0004 |     0.0050    |    0.0050    |    -0.77     | [0.1595, 0.1785] |
+--------------+------------+-----------

In [19]:
wb_rad.summary()

+---------------------------------------------------------------------------------------------------------------------+
|        Wild Bootstrap with Rademacher results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI       |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|     Var      | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|    const     |   6.2070   |     6.2042    | 0.0028 |     0.0454    |    0.0446    |     1.61     | [6.1227, 6.3006] |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| tanass_clean |   0.1693   |     0.1694    | 0.0001 |     0.0050    |    0.0051    |    -2.73     | [0.1592, 0.1791] |
+--------------+------------+-----------

In [20]:
wb_webb4.summary()

+---------------------------------------------------------------------------------------------------------------------+
|          Wild Bootstrap with Webb4 results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI          |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|     Var      | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|    const     |   6.2070   |     6.2091    | 0.0021 |     0.0454    |    0.0446    |     1.74     | [6.1190, 6.2954] |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| tanass_clean |   0.1693   |     0.1692    | 0.0001 |     0.0050    |    0.0051    |    -3.07     | [0.1599, 0.1799] |
+--------------+------------+-----------

In [21]:
wb_webb6.summary()

+---------------------------------------------------------------------------------------------------------------------+
|          Wild Bootstrap with Webb6 results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI          |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|     Var      | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|    const     |   6.2070   |     6.2093    | 0.0023 |     0.0454    |    0.0445    |     2.00     | [6.1194, 6.2943] |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| tanass_clean |   0.1693   |     0.1692    | 0.0001 |     0.0050    |    0.0051    |    -2.69     | [0.1595, 0.1794] |
+--------------+------------+-----------

In [22]:
wb_cont_mam.summary()

+---------------------------------------------------------------------------------------------------------------------+
|       Wild Bootstrap with Mammen Cont results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI       |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|     Var      | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|    const     |   6.2070   |     6.2049    | 0.0021 |     0.0454    |    0.0435    |     4.14     | [6.1300, 6.2952] |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| tanass_clean |   0.1693   |     0.1692    | 0.0001 |     0.0050    |    0.0051    |    -2.93     | [0.1592, 0.1791] |
+--------------+------------+-----------

In [23]:
wb_mam.summary()

+---------------------------------------------------------------------------------------------------------------------+
|          Wild Bootstrap with Mammen results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI         |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|     Var      | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|    const     |   6.2070   |     6.2068    | 0.0002 |     0.0454    |    0.0462    |    -1.75     | [6.1173, 6.3003] |
+--------------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| tanass_clean |   0.1693   |     0.1692    | 0.0000 |     0.0050    |    0.0052    |    -3.26     | [0.1591, 0.1788] |
+--------------+------------+-----------

Same output for wb_mam_np as in the case for wb_mam, but the difference is the independent variables' name. wb_mam was constructed with pd.DataFrame the input which holds information about the column name. 
When it comes to wb_mam_np, it was constructed with np.ndarray as the input which DOES NOT hold info about the column name originally. Because of this, x1, x2, ...., xn were generated as alternative independent variable names. 

In [24]:
wb_mam_np.summary()

+--------------------------------------------------------------------------------------------------------------+
|      Wild Bootstrap with Mammen results with 10000 obs and 1000 BS reps using HC3 SE-s and 95.00% BC CI      |
+-------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|  Var  | OLS Params | Avg BS Params |  Bias  | OLS Params SE | BS Params SE | % of SE Diff |        CI        |
+-------+------------+---------------+--------+---------------+--------------+--------------+------------------+
| const |   6.2070   |     6.2068    | 0.0002 |     0.0454    |    0.0462    |    -1.75     | [6.1173, 6.3003] |
+-------+------------+---------------+--------+---------------+--------------+--------------+------------------+
|   x1  |   0.1693   |     0.1692    | 0.0000 |     0.0050    |    0.0052    |    -3.26     | [0.1591, 0.1788] |
+-------+------------+---------------+--------+---------------+--------------+--------------+---

## Some useful methods were implemented (let's use it on wb_mam object of Wild Bootstrap with Mammen)

- The common in the following three methods is that either a string (if one wishes to capture one variable or ci) or 1-D like array can be provided to 'which_var' and/or 'which_ci' (the latter if exists in that object class).

### get_ci() method

#### Vanila version is when only the actual CI was used with all independent variables

In [25]:
wb_mam.get_ci()

bc                     
                              lwb                  upb
const                     6.11731              6.30032
tanass_clean              0.15910              0.17883
tax                       0.51172              0.54700

#### However, any combination of ('bc', 'bca', 'percentile) CI types could be selected and any combinations of independent variables can be chosen with 'which_ci' and 'which_var' optional arguments, respectively.

Please note that if choosing 'bca' the calculation can take a while as it uses jacknife resampling for calculating the acceleration factor

In [26]:
wb_mam.get_ci(which_ci='all', which_var='all')

bc                                       bca  \
                              lwb                  upb                  lwb   
const                     6.11731              6.30032              6.11736   
tanass_clean              0.15910              0.17883              0.15921   
tax                       0.51172              0.54700              0.51145   

                                            percentile                       
                              upb                  lwb                  upb  
const                     6.30035              6.11720              6.30024  
tanass_clean              0.17886              0.15943              0.17903  
tax                       0.54688              0.51167              0.54698

In [27]:
wb_mam.get_ci(which_ci=['bc', 'percentile'], which_var=['tax', 'const'])

bc                                percentile  \
                       lwb                  upb                  lwb   
tax                0.51172              0.54700              0.51167   
const              6.11731              6.30032              6.11720   

                            
                       upb  
tax                0.54698  
const              6.30024

In [28]:
wb_mam.get_ci(which_ci=['bca', 'bc'], which_var='tanass_clean')

bc                                       bca  \
                              lwb                  upb                  lwb   
tanass_clean              0.15910              0.17883              0.15921   

                                   
                              upb  
tanass_clean              0.17886

In [29]:
wb_mam.get_ci(which_ci='bc', which_var='tanass_clean')

bc                     
                              lwb                  upb
tanass_clean              0.15910              0.17883

### get_all_se() method

The following SE-s are calculated: 
- bootstrapped - standard error of the bootstrapped parameters
- constant - model-based OLS Standard Errors, that is, constant variance is assumed 
- HC0, HC1, HC2, HC3, HC4, HC4m, HC5 - Heteroskedasticity-Consistent Standard Errors (HCE) using sandwich estimators 

#### Vanila version is when using all indepencdent variables. 

In [30]:
wb_mam.get_all_se()

,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5,bootstrapped
const,0.03532,0.04536,0.04536,0.04537,0.04538,0.04538,0.04538,0.04538,0.04617
tanass_clean,0.00391,0.00499,0.00499,0.00499,0.00499,0.00499,0.00499,0.00499,0.00515
tax,0.00657,0.00910,0.00910,0.00911,0.00911,0.00911,0.00911,0.00911,0.00934


#### A subset of independent variables can be chosen with 'which_var' argument to calculate the above-mentioned 9 SE-s

In [31]:
wb_mam.get_all_se(which_var=['tanass_clean', 'tax'])

,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5,bootstrapped
tanass_clean,0.00391,0.00499,0.00499,0.00499,0.00499,0.00499,0.00499,0.00499,0.00515
tax,0.00657,0.00910,0.00910,0.00911,0.00911,0.00911,0.00911,0.00911,0.00934


In [32]:
wb_mam.get_all_se(which_var='tax')

,constant,hc0,hc1,hc2,hc3,hc4,hc4m,hc5,bootstrapped
tax,0.00657,0.00910,0.00910,0.00911,0.00911,0.00911,0.00911,0.00911,0.00934


### get_bootstrap_params() method

#### Vanila version: Returning a dataframe capturing the parameter estimate of ALL each independent variables in each (wild) bootstrap.

In [33]:
wb_mam.get_bootstrap_params()

,const,tanass_clean,tax
0,6.12845,0.17665,0.53513
1,6.14831,0.17589,0.52740
2,6.16231,0.17062,0.53366
3,6.26950,0.16372,0.52545
4,6.22074,0.16960,0.52395
...,...,...,...
995,6.17476,0.17185,0.52916
996,6.19431,0.17210,0.52284
997,6.20963,0.17164,0.52893
998,6.19527,0.16879,0.53375


#### As usual, the desired independent variable can be chosen with 'which_var' argument

In [34]:
wb_mam.get_bootstrap_params(which_var='tax')

,tax
0,0.53513
1,0.52740
2,0.53366
3,0.52545
4,0.52395
...,...
995,0.52916
996,0.52284
997,0.52893
998,0.53375


In [35]:
wb_mam.get_bootstrap_params(which_var=('const', 'tanass_clean'))

,const,tanass_clean
0,6.12845,0.17665
1,6.14831,0.17589
2,6.16231,0.17062
3,6.26950,0.16372
4,6.22074,0.16960
...,...,...
995,6.17476,0.17185
996,6.19431,0.17210
997,6.20963,0.17164
998,6.19527,0.16879


### bp_test() and white_test() methods

#### Vanila version: for bp_test() returning the robust version (proposed by Roger Koenker) of this test when the presence of the heteroscedasticity is examined on the residual of the OLS on the original sample data. For white_test(), there are no optional arguments

In [36]:
wb_mam.bp_test()

(432.43914611970746,
 1.2503527125935506e-94,
 225.92457000184137,
 1.083837549448663e-96)

In [37]:
wb_mam.white_test()

(1437.6088470921256, 9.758682033770634e-309, 335.59463848972536, 0.0)

#### The non-robust version of bp_test() can be achieved by setting bp_test(robust = False).

In [38]:
wb_mam.bp_test(robust = False)

(1035.8816259850282,
 1.1512322968569012e-225,
 225.92457000184234,
 1.083837549446753e-96)

At each test, the first value is the calculated LM-test stats, the second is the 'LM p-value' corresponding to the LM-test stats, the third value is the F-test stats and last one is the 'F p-value'.

As it can be seen the p-values are 0, so we can rejcet the null hypothesis of homoscedasticity, and accept the alternative hypothesis that the variance of the residuals is heteroscedastic.

### Some built-in attributes

In [39]:
wb_mam.indep_varname

['const', 'tanass_clean', 'tax']

#### wb_mam.bs_params and wb_mam.get_bootstrap_params() basically output the same thing with the exception that the latter one is transposed and is in pd.DataFrame.

In [40]:
wb_mam.bs_params

array([[6.12844641, 6.14830663, 6.16230646, ..., 6.20962879, 6.19526923,
        6.25778397],
       [0.17665437, 0.17588706, 0.17061541, ..., 0.17164358, 0.16878517,
        0.17449918],
       [0.53512931, 0.52739577, 0.53366326, ..., 0.5289288 , 0.53375046,
        0.51449554]])

In [41]:
wb_mam.orig_params

array([6.20695725, 0.16929278, 0.52929771])

In [42]:
wb_mam.bs_mean

array([6.20678835, 0.16924469, 0.52938636])

In [43]:
wb_mam.orig_se

array([0.0453789 , 0.00499137, 0.00910921])

In [44]:
wb_mam.bs_se

array([0.04617403, 0.00515419, 0.00933865])

In [45]:
wb_mam.bs_ci

array([[6.11731089, 6.3003165 ],
       [0.15909888, 0.17882641],
       [0.51172172, 0.54700073]])